In [50]:
import pandas as pd
import matplotlib.pyplot as plt

import torch
import random
import csv
import os
from PIL import Image

from Config import Config
from Models import PreTrainedKPD, PreTrainedConf, Tools

In [51]:
"""
Load the model weights.
"""

device = "mps" if torch.backends.mps.is_available() else "cpu"

kpd_model = PreTrainedKPD.KeypointDetectionModel().to(device)
kpd_model.load_state_dict(torch.load("Models/Saves/pretrainedKPD_weights00166.pth", map_location=device))

conf_model = PreTrainedConf.ConfidenceModel().to(device)
conf_model.load_state_dict(torch.load("Models/Saves/pretrainedConf_weights00166.pth", map_location=device))

/home/wene/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wene/miniconda3/envs/mlenv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_58269/923896920.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/mai

<All keys matched successfully>

In [52]:
def get_image_shape(file_path):
    with Image.open(file_path) as img:
        return img.size

In [53]:
notes = pd.read_csv('CSVs/combined.csv')
preds = pd.read_csv('predictions_17december.csv')

preds['Image Name'] = preds['Image Name'].map(lambda s : os.path.splitext(s)[0])
notes['Image Name'] = notes['Image Name'].map(lambda s : os.path.splitext(s)[0])

notes = notes.groupby(['Image Name']).mean()
notes['shape'] = notes.index.map(lambda s : get_image_shape(
    os.path.join(Config.images_folder_path, s + '.png')
    )
)

In [54]:
notes

,x1,y1,x2,y2,shape
Image Name,,,,,
14d618c_p_1,349.0,221.0,668.0,374.0,"(1084, 808)"
14d709d-1_p_1,262.0,212.0,555.0,317.0,"(1028, 808)"
14d806b_p_1,255.0,197.0,551.0,325.0,"(1042, 808)"
14l327b-3_p_1,296.0,148.0,585.0,238.0,"(1146, 808)"
14l358c-3_p_1,414.0,159.0,683.0,284.0,"(1049, 808)"
...,...,...,...,...,...
usnment00755077_p_1,310.0,192.0,578.0,234.0,"(920, 808)"
usnment00757164_p_1,324.0,204.0,558.0,340.0,"(1147, 808)"
usnment00921167_p_1,256.0,201.0,738.0,261.0,"(1219, 808)"


In [49]:
preds.set_index('Image Name').join(notes, lsuffix='_caller', rsuffix='_other')

,x1_caller,y1_caller,x2_caller,y2_caller,Error estimate,x1_other,y1_other,x2_other,y2_other,shape
Image Name,,,,,,,,,,
casent0132636_p_1,660.017759,240.663123,427.678719,199.314543,0.279193,NaN,NaN,NaN,NaN,NaN
casent0282351_p_1,677.203557,237.956789,437.371318,171.138983,0.202611,NaN,NaN,NaN,NaN,NaN
casent0179888_p_1,545.244099,267.285892,624.458198,334.750615,0.424048,NaN,NaN,NaN,NaN,NaN
casent0911031_p_1,583.721092,248.147129,408.531846,152.017967,0.412925,NaN,NaN,NaN,NaN,NaN
casent0903450_p_1,972.899839,340.098146,509.674106,142.943604,0.346776,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
casent0922995_p_1,645.841501,314.579266,292.707428,135.464727,0.352304,NaN,NaN,NaN,NaN,NaN
casent0173971_p_1,631.541429,209.871650,374.023497,196.564672,0.264337,NaN,NaN,NaN,NaN,NaN
casent0102561_p_1,784.063263,369.925638,394.595131,175.042636,0.329181,NaN,NaN,NaN,NaN,NaN


In [40]:
df

,x1_predicted,y1_predicted,x2_predicted,y2_predicted,Error estimate,x1_real,y1_real,x2_real,y2_real,shape
Image Name,,,,,,,,,,
casent0132636_p_1,660.017759,240.663123,427.678719,199.314543,0.279193,NaN,NaN,NaN,NaN,NaN
casent0282351_p_1,677.203557,237.956789,437.371318,171.138983,0.202611,NaN,NaN,NaN,NaN,NaN
casent0179888_p_1,545.244099,267.285892,624.458198,334.750615,0.424048,NaN,NaN,NaN,NaN,NaN
casent0911031_p_1,583.721092,248.147129,408.531846,152.017967,0.412925,NaN,NaN,NaN,NaN,NaN
casent0903450_p_1,972.899839,340.098146,509.674106,142.943604,0.346776,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
casent0922995_p_1,645.841501,314.579266,292.707428,135.464727,0.352304,NaN,NaN,NaN,NaN,NaN
casent0173971_p_1,631.541429,209.871650,374.023497,196.564672,0.264337,NaN,NaN,NaN,NaN,NaN
casent0102561_p_1,784.063263,369.925638,394.595131,175.042636,0.329181,NaN,NaN,NaN,NaN,NaN


In [ ]:
errors = (keypoints - outputs)**2
mse_per_vector = torch.mean(errors, dim=1, keepdim=True)

mse_per_vector = torch.clamp(mse_per_vector, max=clamp_threshold)
mse_per_vector_np = mse_per_vector.cpu().numpy()

# Step 1: Normalize by mean and variance
mean = mse_per_vector.mean()
std = mse_per_vector.std()
normalized_vector = (mse_per_vector - mean) / std

# Step 2: Scale to range [0, 1]
min_val = normalized_vector.min()
max_val = normalized_vector.max()
scaled_errors = (normalized_vector - min_val) / (max_val - min_val)